In [1]:
# OpenAI Gym
import gym_tetris as gym
import gym.spaces
import numpy as np
import keras

# Rendering tools
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display
from IPython.display import clear_output
import cv2

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


Using TensorFlow backend.


In [2]:
env = gym.make('Tetris-v0')
downsize = (26,51)
dsize = (51,26)
observation=env.reset()

In [3]:
def resize(img,downsize):
    resized = cv2.resize(img,
                         dsize=downsize,
                         interpolation=cv2.INTER_CUBIC)
    bwarray=[]
    for x in range(resized.shape[0]):
        row=[]
        for y in range(resized.shape[1]):
            r,g,b = observation[x][y]
            if int(r) > 0 or int(g) > 0 or int(b) > 0:
                row.append([1])
            else:
                row.append([0])
        bwarray.append(row)
    bwarray=np.array(bwarray)
    return bwarray

In [4]:
resized_observation = resize(observation[17:423,7:212],downsize)
in_size = resized_observation.shape
obs_size = resized_observation.size
display(in_size)
display(obs_size)

(51, 26, 1)

1326

In [5]:
def make_model(state, action_size):
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(128, kernel_size=(8, 8), activation='relu',
                                    input_shape=[state.shape[0],
                                                 state.shape[1],
                                                 state.shape[2]]))
    model.add(keras.layers.Conv2D(128, (8, 8), activation='relu'))
    model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(20, activation='relu'))
    model.add(keras.layers.Dense(action_size, activation='linear'))
    model.compile(loss='mse',optimizer=keras.optimizers.Adam(lr=0.001))
    return model

#model = make_model(resized_observation, env.action_space.n)
#model.summary()
model = make_model(resize(observation,downsize), 
                   env.action_space.n)
model.summary()

The history saving thread hit an unexpected error (OperationalError('disk I/O error',)).History will not be written to the database.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 44, 19, 128)       8320      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 37, 12, 128)       1048704   
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 18, 6, 128)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 18, 6, 128)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 13824)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                276500    
_________

In [6]:
class ReplayMemory:
    def __init__(self, memory_size, state_size, action_size):
        self.state_size = [state_size.shape[0], state_size.shape[1], state_size.shape[2]]
        self.action_size = action_size
        self.size = 0
        self.maxsize = memory_size
        self.current_index = 0
        #self.current_state = np.zeros([memory_size, self.state_size[0], self.state_size[1], 
        #                               self.state_size[2]])
        self.current_state = np.zeros([memory_size, 
                                       self.state_size[0], 
                                       self.state_size[1], 
                                       self.state_size[2]])
        self.action = [0]*memory_size # Remember, actions are integers...
        self.reward = np.zeros([memory_size])
        self.next_state = np.zeros([memory_size, 
                                    self.state_size[0], 
                                    self.state_size[1],
                                    self.state_size[2]])
        self.done = [False]*memory_size # Boolean (terminal transition?)
        
    def remember(self, current_state, action, reward, next_state, done):
        # Stores a single memory item
        self.current_state[self.current_index,:] = current_state
        self.action[self.current_index] = action
        self.reward[self.current_index] = reward
        self.next_state[self.current_index,:] = next_state
        self.done[self.current_index] = done
        self.current_index = (self.current_index+1)%self.maxsize
        self.size = max(self.current_index,self.size)
    
    def replay(self, model, target_model, num_samples, sample_size, gamma):
        # Run replay!
        
        # Can't train if we don't yet have enough samples to begin with...
        if self.size < sample_size:
            return
        
        for i in range(num_samples):
            # Select sample_size memory indices from the whole set
            current_sample = np.random.choice(self.size,sample_size,replace=False)
            
            # Slice memory into training sample
            current_state = self.current_state[current_sample,:]
            action = [self.action[j] for j in current_sample]
            reward = self.reward[current_sample]
            next_state = self.next_state[current_sample,:]
            done = [self.done[j] for j in current_sample]
            if current_state.size == obs_size:
                model_targets = model.predict(resize(current_state, dsize))
                # Create targets from argmax(Q(s+1,a+1))
                # Use the target model!
                print(next_state.size)
                targets = reward + gamma*np.amax(target_model.predict(next_state,axis=1))

                # Absorb the reward on terminal state-action transitions
                targets[done] = reward[done]
                # Update just the relevant parts of the model_target vector...
                model_targets[range(sample_size),action] = targets

                # Update the weights accordingly
                model.fit(resize(current_state, downsize),model_targets,
                          epochs=1,verbose=0,batch_size=sample_size)
            else:
                continue

            # Once we have finished training, update the target model
            target_model.set_weights(model.get_weights())

In [7]:
from collections import deque

# Hyperparameters
gamma = 0.95
epsilon = 1.0
epsilon_decay = 0.994
epsilon_min = 0.01
episodes = 3000

replay_iterations = 100
replay_sample_size = 256

# Peformance stats
times_window = deque(maxlen=100)
mean_times = deque(maxlen=episodes)

# Initialize the environment and agent data structures
env = gym.make('Tetris-v0')
model = make_model(resize(env.reset(),dsize), env.action_space.n)
target_model = make_model(resize(env.reset(),dsize), env.action_space.n)
memory = ReplayMemory(8000, resize(env.reset(),dsize), env.action_space.n)

In [ ]:
model.load_weights('weights_v2.h5')

In [10]:
#Run this when display surface quit
env = gym.make('Tetris-v0')
observation = env.reset()

In [ ]:
done = False
mean_time = 900
# Perform the training!
for episode in range(episodes):
    current_state = env.reset()
    current_state = current_state[17:423,7:212]
    reward_earned = 0
    time = 0
    while not done:
        time += 1
        Q = model.predict(np.expand_dims(resize(current_state, dsize), axis=0)) # Compute Q
        action = np.random.choice(env.action_space.n) if np.random.random() < epsilon else np.argmax(Q)
        next_state, reward, done, info = env.step(action) # take action!
        next_state = resize(next_state[17:423,7:212], dsize)
        if done:
            reward -= 10.0
            reward_earned -= 10.0
            if time < mean_time:
                reward -= 20.0
                reward_earned -= 20.0
            else:
                reward += 10.0
                reward_earned += 10.0
        reward += 0.01*pow(time,0.1)
        reward_earned += 0.01*pow(time,0.1)
        if current_state.size == obs_size:
            resize_state = resize(current_state,dsize)
            memory.remember(resize_state, action,reward,next_state,done) # Store in memory...
        current_state = next_state # Transition to next state!
    epsilon = epsilon * epsilon_decay if epsilon > epsilon_min else epsilon_min
    times_window.append(time)
    mean_time = np.mean(times_window)
    mean_times.append(mean_time)
    print('\rEpisode %d/%d - time: %d, mean-time: %d, epsilon: %f, reward: %g'%(episode+1,episodes,time,mean_time,epsilon,reward_earned),end='')
    done = False
    
    # Training...
    memory.replay(model,target_model,replay_iterations,replay_sample_size,gamma)
print()

Episode 40/3000 - time: 932, mean-time: 893, epsilon: 0.786059, reward: 16.7924

In [ ]:
model.save_weights('weights_v2.h5')

In [ ]:
plt.plot(mean_times)
plt.title("Tetris Q-Learning Performance")
plt.xlabel("Episode Number")
plt.ylabel("Life Span")
plt.show()